# Web Scraping

In [1]:
###Load modules
#!pip install webdriver-manager
#!pip install awscli
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table


page = '7'

for i in range(1,int(page)+1): 
      page = browser.find_element(By.XPATH,'/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a['+str(i)+']')
      for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
...,...,...,...,...,...,...
107,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
108,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
109,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
110,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY


# Fixing the DataFrame 

In [2]:
# Convert None values to NaN
df.replace(to_replace=[None], value=pd.NA, inplace=True)

# Drop rows where all columns are NaN
df.dropna(how='all', inplace=True)

# Reset index 
df.reset_index(drop=True, inplace=True)

df.head()

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY


# Loading the corrected Data into s3

In [3]:
from io import StringIO
from aws_credentials import AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY


###LOAD THE FILE INTO S3####
# Prepare csv file name
bucket_name = 'm-10-database-update-bucket'  # Specify the S3 bucket name
filename = 'charities_bureau_scrape_'  # Name of your group
timestamp = time.strftime("%Y%m%d%H%M%S")  # Timestamp
filenames3 = f"{filename}{timestamp}.csv"  # File name

# Write DataFrame to CSV in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)


# Connect to S3 with explicit access key and secret key
access_key = AWS_ACCESS_KEY_ID
secret_key = AWS_SECRET_ACCESS_KEY
s3_resource = boto3.resource('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)

# Upload CSV to S3 bucket
s3_resource.Object(bucket_name, filenames3).put(Body=csv_buffer.getvalue())

# Print success message
print("Successfully uploaded file to location:", f"s3://{bucket_name}/{filenames3}")

Successfully uploaded file to location: s3://m-10-database-update-bucket/charities_bureau_scrape_20240416215624.csv
